In [21]:
using MotionCaptureJointCalibration
using RigidBodyDynamics
using StaticArrays
using ValkyrieRobot
using ForwardDiff
using Ipopt

Create a `Mechanism` (we'll use the NASA Valkyrie robot) and `MechanismState`:

In [22]:
T = Float64
val = Valkyrie()
mechanism = val.mechanism
remove_fixed_tree_joints!(mechanism)
state = MechanismState{T}(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

Let's generate a random calibration problem, as well as its ground truth solution:

In [23]:
using MotionCaptureJointCalibration.SyntheticDataGeneration
bodies_with_markers = findbody.(mechanism, ["leftFoot", "pelvis"])
srand(1)
body_weights = Dict(b => rand() for b in bodies_with_markers)
problem, groundtruth = generate_calibration_problem(state, body_weights);

In [24]:
problem

CalibrationProblem{Float64} with:
* 25 poses
* 6 calibration parameters
* 8 markers attached to 2 bodies



In [25]:
groundtruth

CalibrationResult{Float64}: Optimal, residual = 0.0. Calibration parameters:
leftHipRoll: [0.00907607]
leftHipPitch: [-0.00809629]
leftKneePitch: [0.000393507]
leftAnklePitch: [-0.00972919]
leftHipYaw: [0.00473574]
leftAnkleRoll: [-0.00393202]

Solve the calibration problem (without using the ground truth result):

In [26]:
solver = IpoptSolver(print_level = 0, max_iter = 10000, derivative_test = "first-order", check_derivatives_for_naninf = "yes", tol = 1e-10)
result = solve(problem, solver)

CalibrationResult{Float64}: Optimal, residual = 1.115199422112139e-9. Calibration parameters:
leftHipRoll: [0.0090898]
leftHipPitch: [-0.00804067]
leftKneePitch: [0.000380741]
leftAnklePitch: [-0.00980915]
leftHipYaw: [0.00472167]
leftAnkleRoll: [-0.00402982]

And visualize the results:

In [27]:
using RigidBodyTreeInspector
using DrakeVisualizer

In [58]:
#NBSKIP
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));

In [59]:
vis = Visualizer()[:valkyrie]
setgeometry!(vis, mechanism, parse_urdf(ValkyrieRobot.urdfpath(), mechanism; package_path = [ValkyrieRobot.packagepath()]))
state = MechanismState{T}(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

In [61]:
inspect!(state, vis, problem, result)

Interact.Options{:SelectionSlider,Bool}(22: "input-11" = false Bool , "", false, "Before cal", Interact.OptionDict(DataStructures.OrderedDict("Before cal"=>false,"After cal"=>true), Dict(false=>"Before cal",true=>"After cal")), Any[], Any[], true, "horizontal")

Interact.Slider{Int64}(24: "input-12" = 1 Int64 , "Pose number", 1, 1:25, "horizontal", true, "d", true)